In [1]:
!cat .env

STAGE=PRODUCTION


In [2]:
import json
from django.contrib.admin.utils import flatten
from helpers.strings import keys_to_snake_case, keys_to_camel_case
from rest_framework_json_api.utils import format_field_names
from tqdm.notebook import tqdm

In [3]:
def generate_select_multiple_export(field):

    result={"cat":[]}    
    values=field["selectOptions"]["options"]

    for value in values:
        if field["apiName"] == "demandOrigin":
            result["cat"].append({"if":[{"in": 
                                     [value["value"], {"var": "formData."+field["apiName"]}]
                                    }, value["name"] + " ", ""]})
        else:
            result["cat"].append({"if":[{"in": 
                                     [value["value"], {"var": "formData."+field["apiName"]}]
                                    }, value["name"] + ", ", ""]})


    return result

In [4]:
def generate_select_export(field):
    result={"if":[]}    
    values=field["selectOptions"]["options"]
    
    for value in values:
        result["if"].append({"==":[{"var": "formData."+field["apiName"]}, value["value"]]})
        result["if"].append(value["name"])
        
    return result

In [16]:
def generate_export(field):
    result={"header": field["displayName"], "key": field["apiName"]}
    
    if field["dataType"]=="timestamp":
        result["isDate"]=True
    elif (field["dataType"]=="select") and ("options" in field["selectOptions"]):
        result["logic"]=generate_select_export(field)
    elif (field["dataType"]=="selectMultiple") and ("options" in field["selectOptions"]):
        result["logic"]=generate_select_multiple_export(field)
    elif field["dataType"]=="jsonLogic":
        result["logic"]=field["logic"]
    elif field["dataType"]=="boolean": # adaptação Eixo SP
        result["logic"]={
                            "if": [{
                                "===": [{
                                    "var": "formData." + field["apiName"]
                                }, True]
                            }, "Sim", "Não"]
                        }
    elif field["dataType"]=="float":
        result["style"]= {
                "num_fmt": "0.000"
                }
    
    
    return result

In [18]:
def generate_array_export(field, number=8):
    result_list=[]
    values=field["innerFields"]
    
    for value in values:
        
        if (value["dataType"] == "select") and ("options" in value["selectOptions"]):            
            for index in range(1, number+1):            
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " "  + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]={}
                result["logic"]["if"]=[]
                values_select=value["selectOptions"]["options"]

                for value_select in values_select:
                    result["logic"]["if"].append({"==":[{"var": "formData." + field["apiName"]+ "."+str(index-1) + "." +value["apiName"]}, value_select["value"]]})
                    result["logic"]["if"].append(value_select["name"])

                result_list.append(result)
        
        elif ("display" in value["apiName"]):
            pass
        elif value["dataType"] == "innerImagesArray":
            pass
        
        elif (value["dataType"] == "selectMultiple") and ("options" in value["selectOptions"]):
            for index in range(1, number+1):            
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " "  + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]={}
                result["logic"]["cat"]=[]
                values_select=value["selectOptions"]["options"]

                for value_select in values_select:
                    result["logic"]["cat"].append({"if":[{"in": 
                                     [value_select["value"], {"var": "formData." + field["apiName"]+ "."+str(index-1) + "." +value["apiName"]}]
                                    }, value_select["name"], ", "]})

                result_list.append(result)
            
        else:
            for index in range(1, number+1): 
                
                result={} 
                result["header"]=field["displayName"] + " - " + value["displayName"] + " " + str(index) 
                result["key"]=value["apiName"]+str(index) 
                
                if value["dataType"] == "boolean": 
                    result["logic"] = { "if": [{ "===": [{ "var": "formData." + field["apiName"] + "." + str(index-1) + "." + value["apiName"] }, True] }, "Sim", "Não"] } 
                else: 
                    result["logic"]= {"var": "formData." + field["apiName"] + "." + str(index-1) + "." + value["apiName"]} 
#                 result["logic"]= {"var": "formData." + field["apiName"] + "." + str(index-1) + "." + value["apiName"]} 
                    if value["dataType"] == "timestamp": result["isDate"] = True 
                result_list.append(result)
    
    
    return result_list

In [19]:
def update_export_data(data, occ=None, filename=None):
    if occ is not None:
        if 'measurement_columns' in occ.form_fields.keys():
            del occ.form_fields['measurement_columns']
        occ.form_fields['measurementColumns']=data
        occ.save()
    if filename is not None:
        with open(filename + '.txt', 'w') as outfile:
            json.dump(data, outfile, indent=4)

In [20]:
def format_keys(obj, format_type="camelize"):
    if isinstance(obj, dict):
        for key, value in obj.items():
            if isinstance(value, dict):
                obj[key] = format_field_names(
                            format_keys(value), format_type
                            )
            elif isinstance(value, list):
                list_temp = value.copy()
                for i, item in enumerate(value):
                    if isinstance(item, dict):
                        list_temp[i] = format_field_names(
                        format_keys(item), format_type
                        )
                    else:
                        list_temp[i] = item
                obj[key] = list_temp
    return obj

In [26]:
# company=Company.objects.get(name='Eixo Sp - Pavimento')
company=Company.objects.get(name='Demo Concessionárias')
company

<Company: f0b6948c-3d75-47dd-94a6-4b8ea4373ce4: Demo Concessionárias>

In [27]:
# occ=OccurrenceType.objects.get(uuid='f6033d7d-9649-420d-9125-2000aefcd25a')
occ=OccurrenceType.objects.filter(company=company, active=True, next_version__isnull=True,occurrence_kind__in=['1999']).order_by('name')
occ.count()

2

In [28]:
for match in tqdm(occ):
    for index, field in enumerate(match.form_fields['fields']):
        match.form_fields['fields'][index]=format_keys(keys_to_camel_case(field))
    match.save()

  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
# columns=[]

# for field in occ.form_fields['fields']:
    
#     if field['dataType'] != 'arrayOfObjects':
#         columns.append(generate_export(field))
#     else:
#         columns.append(generate_array_export(field))
    

# columns=flatten(columns)

# update_export_data(columns, occ=occ)

In [29]:
for match in tqdm(occ):
        
    try:
        columns=[]

        for field in match.form_fields['fields']:
            if field['dataType'] != 'arrayOfObjects':
                columns.append(generate_export(field))
            else:
                columns.append(generate_array_export(field, 6))


        columns=flatten(columns)
        update_export_data(columns, occ=match)
    
    except Exception as e:
        print(f'{match.uuid} - {e}')

  0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
occ[0].form_fields['measurementColumns']

[{'header': 'Código de Edital', 'key': 'contratualCode', 'logic': 'b.1.3.'},
 {'header': 'CSP', 'key': 'csp', 'logic': '7'},
 {'header': 'Origem',
  'key': 'demandOrigin',
  'logic': {'cat': [{'if': [{'in': ['1', {'var': 'formData.demandOrigin'}]},
      'Interna ',
      '']},
    {'if': [{'in': ['2', {'var': 'formData.demandOrigin'}]}, 'KCOR ', '']},
    {'if': [{'in': ['3', {'var': 'formData.demandOrigin'}]},
      'ARTESP DIN ',
      '']},
    {'if': [{'in': ['4', {'var': 'formData.demandOrigin'}]},
      'ARTESP DOP ',
      '']}]}},
 {'header': 'Código ARTESP', 'key': 'artespCode'},
 {'header': 'Recebido da ARTESP em', 'key': 'receivedDate', 'isDate': True},
 {'header': 'Cód KCOR', 'key': 'kcorCode'},
 {'header': 'Cód interno', 'key': 'internCode'},
 {'header': 'Lote',
  'key': 'lot',
  'logic': {'if': [{'==': [{'var': 'formData.lot'}, '1']},
    'Lote 1',
    {'==': [{'var': 'formData.lot'}, '2']},
    'Lote 2',
    {'==': [{'var': 'formData.lot'}, '3']},
    'Lote 3',
    {'==